In [4]:
import tensorflow as tf
import numpy as np
from bunch import Bunch
import os, sys
sys.path.append("..")
from data_generator import hvd_distributed_tf_data_utils as tf_data_utils
import horovod.tensorflow as hvd
epoch = 2
hvd.init()
sess_config = tf.ConfigProto()
sess_config.gpu_options.visible_device_list = str(hvd.local_rank())

os.environ["CUDA_VISIBLE_DEVICES"] = ""

graph = tf.Graph()
with graph.as_default():
    name_to_features = {
                    "input_ids":
                            tf.FixedLenFeature([128], tf.int64),
                    "input_mask":
                            tf.FixedLenFeature([128], tf.int64),
                    "segment_ids":
                            tf.FixedLenFeature([128], tf.int64),
                    "label_ids":
                            tf.FixedLenFeature([], tf.int64),
            }

    def train_input_fn(input_file, _parse_fn, name_to_features,
            params):
        dataset = tf.data.TFRecordDataset(input_file, buffer_size=params.get("buffer_size", 100))
        
        print("==hvd size {}, rank {}==".format(hvd.size(), hvd.rank()))
        dataset = dataset.shard(hvd.size(), hvd.rank())
        dataset = dataset.map(lambda x:_parse_fn(x, name_to_features))
        dataset = dataset.shuffle(
                                buffer_size=params.get("buffer_size", 1024)+3*params.get("batch_size", 32),
                                seed=np.random.randint(0,1e10,1)[0],
                                reshuffle_each_iteration=True)
        dataset = dataset.batch(params.get("batch_size", 32))
        dataset = dataset.repeat(params.get("epoch", 100))
        iterator = dataset.make_one_shot_iterator()
        print(dataset.output_shapes)
        features = iterator.get_next()
        return features

    params = Bunch({})
    params.epoch = epoch
    params.batch_size = 32
    jd_test = "/data/xuht/jd_comment/test.tfrecords"
    input_fn = train_input_fn(jd_test, tf_data_utils._decode_record, name_to_features, params)
    
    sess = tf.Session(config=sess_config)
    
    init_op = tf.group(
                tf.local_variables_initializer())
    sess.run(init_op)

    sess.run(hvd.broadcast_global_variables(0))
    
    i = 0
    cnt = 0
    while True:
        try:
            features = sess.run(input_fn)
            i += 1
            cnt += 1
        except tf.errors.OutOfRangeError:
            print("End of dataset")
            break
    print(i)
    print(hvd.rank())

==hvd size 1, rank 0==
{'label_ids': TensorShape([Dimension(None)]), 'segment_ids': TensorShape([Dimension(None), Dimension(128)]), 'input_mask': TensorShape([Dimension(None), Dimension(128)]), 'input_ids': TensorShape([Dimension(None), Dimension(128)])}
End of dataset
538
0


In [4]:
1033*32

33056